<img src="img/header.png"></img>

<h2> Download Chrome Driver dan Chrome Binary </h2>

In [ ]:
# untuk Ubuntu 16.04  
# download chrome driver
!curl -SL https://chromedriver.storage.googleapis.com/2.37/chromedriver_linux64.zip > chromedriver.zip
!unzip chromedriver.zip
!rm chromedriver.zip
!sudo mv chromedriver /usr/local/bin

# download chrome binary
!curl https://gist.githubusercontent.com/xxxxlr/18ef864b6374f04939890d9cc186571f/raw/990d54467543d17e5c1874dccc3b7970a63d1ca4/install.sh | bash
# !curl https://gist.githubusercontent.com/xxxxlr/18ef864b6374f04939890d9cc186571f/raw/990d54467543d17e5c1874dccc3b7970a63d1ca4/start-chrome.sh | bash

#install selenium
!pip install selenium

In [57]:
# untuk redhat, centos
# download chrome driver
#!curl -SL https://chromedriver.storage.googleapis.com/2.37/chromedriver_linux64.zip > chromedriver.zip
#!unzip chromedriver.zip
#!rm chromedriver.zip
#!sudo mv chromedriver /usr/local/bin

# download chrome binary
#!curl https://intoli.com/install-google-chrome.sh | bash
    
#install selenium
#!pip install selenium

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3632k  100 3632k    0     0  29.8M      0 --:--:-- --:--:-- --:--:-- 29.8M
Archive:  chromedriver.zip
replace chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9526  100  9526    0     0   387k      0 --:--:-- --:--:-- --:--:--  387k
Working in /tmp/google-chrome-installation
/tmp/google-chrome-installation /home/ec2-user/SageMaker
Configuring the Google Chrome repo in /etc/yum.repos.d/google-chrome.repo
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper
amzn-main                                                | 2.1 kB     00:00     
amzn-updates                                             | 2.5 kB     00:00     
google-chrome          

<h2> Konfigurasi Chrome Driver </h2>

In [53]:
from selenium import webdriver
import sys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

<h2> Import Library Python </h2>

In [81]:
#Import Library Python
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

<h2> Class untuk melakukan query ke Tweeter </h2>

In [62]:
#Class untuk mendapatkan tweet dari twitter
class TweetsCrawler(object):
    #Pause (untuk menunggu loading data)
    SCROLL_PAUSE_TIME = 0.3
    
    def __init__(self):
        #Browser (menggunakan chrome driver)
        self.browser = webdriver.Chrome('chromedriver',options=chrome_options)
        #Base URL untuk mencari tweet
        self.base_url = 'https://twitter.com/search?q='
        self.proceeding_url = ""
    def get_data(self, query, scroll=0):
        ''' 
        Fungsi untuk mencari tweet.
        
        Parameter Doc.
          @Param: query
          @Desc: string query yang ingin dicari
          
          @Param: scroll
          @Default: 0
          @Desc: berapa banyak kali scroll yang diinginkan. (mensimulasikan scroll layaknya pada web browser)
        '''
        
        #Buka URL
        self.browser.get(self.base_url+query+self.proceeding_url)

        #dapatkan html tag <body>
        body = self.browser.find_element_by_tag_name('body')
        
        #Simulasikan scroll
        for _ in range(scroll):
            #Sleep untuk menunggu loading data
            time.sleep(TweetsCrawler.SCROLL_PAUSE_TIME)
            #Scroll ke bawah
            self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        #Daftar tweet
        all_tweets = []
        
        #Parsing HTML dengan bs4
        html = BeautifulSoup(body.get_attribute('innerHTML'), 'html.parser')
        #Select result
        result = html.select('#timeline li.stream-item')
        #Untuk semua Tweet dalam result
        for tweet in result:
            #Ambil tweet id
            tweet_id = tweet['data-item-id']
            #Ambil tweet text
            tweet_text = tweet.select('p.tweet-text')[0].get_text()
            #masukkan ke daftar tweet
            all_tweets.append({"id": tweet_id, "text": tweet_text})
            
        #Kembalikan tweet sebagai Pandas DataFrame
        return pd.DataFrame(all_tweets)


<h2> Class untuk HashTag Crawler </h2>

In [98]:
def flatten(l):
    return [item for sublist in l for item in sublist]

class HashTagCrawler(TweetsCrawler):
    def __init__(self):
        TweetsCrawler.__init__(self)
        self.base_url = 'https://twitter.com/hashtag/'
    def get_data(self, query, scroll=0):
        data = TweetsCrawler.get_data(self, query, scroll)
        hashtags = data.text.str.findall(r'#.*?(?=\s|$)')
        return pd.DataFrame(set(flatten(hashtags)))

<h2> Query Tweet di Twitter </h2>

In [87]:
#Buat object Selenium Client
tc = TweetsCrawler()

#Ambil Tweets dengan query "Big Data Analytics"
tweets_df = tc.get_data('Big Data Analytics', 10)

#Tampilkan Hasil
print(tweets_df)

                     id                                               text
0   1150877677315903488  Peter Theil thinks @Google $googl should be in...
1   1149722914922618880  2019 top digital transformation tech investmen...
2   1150696313438294016  How #Blockchain Will Improve Your Big Data\n#B...
3   1151027286679810048  The future of farming will rely on big data to...
4   1151129889929605120  [EBOOK]  Smart Analytics: The smart way to pro...
5   1150990958814793728  How can you find value in 4 steps with Big Dat...
6   1148935019143950336  What is the #InternetOfThings? Why is it so im...
7   1151104689422757888  Germany’s ‘BIG DATA’ technology is largely dri...
8   1151050236284067840  How to track objects on the factory floor or s...
9   1148481165784825856  My Big Data professor mentioned Amitab Bachcha...
10  1147787598976487424  Searching for #BigData Capital of the World ht...
11  1151139381677215745  Hottest Big Data Analytics Trends that will do...
12  1150079999220822019  

<h2> Crawling HashTag </h2>

In [99]:
#Buat object Selenium Client
hc = HashTagCrawler()

#Ambil Tweets dengan query "Big Data Analytics"
hashtag_df = hc.get_data('BigData', 10)

#Tampilkan Hasil
print(hashtag_df)

                                                    0
0                                               #IoT?
1                                       #CyberAttacks
2                                                #iot
3                                     #DataScientists
4               #Healthcarepic.twitter.com/QkdBZw1p1N
5             #Infographicspic.twitter.com/enRVpCkfge
6                                      #surveillance,
7                                            #Machine
8                                      #SatelliteData
9                                          #PokemonGO
10                                    #cloudcomputing
11                             #Digitaltransformation
12                                     #DataAnalytics
13   #SocialPrescribing2019pic.twitter.com/6IM1NbzjUn
14                  #hadooppic.twitter.com/mSGKRGb8Pn
15                #Mobilitypic.twitter.com/1umcyjCwK2
16                                  #DataMonetization
17                          

<img src="img/thumbs-up.png"></img>